In [63]:
import pandas as pd
import numpy as np

In [64]:
df = pd.read_csv('tabela.csv', sep=';')
df.shape

(10757, 31)

In [65]:
df = df.dropna(subset=['nome_alimento'])
# df

In [66]:
# df.isnull().sum()

In [67]:
df['alimento_verificado'] = 'TRUE'
df['alimento_ativo'] = 'TRUE'
df['dtt_criacao_alimento'] = '2024-08-29 00:00:00'
df['unidade_medida'] = 'GRAMA'
df['porcao_padrao'] = 100
# df.head()

In [68]:
for col in df.columns:
    df[col] = df[col].replace('-', '')

In [69]:
df['kcal'] = df['kcal'].replace('', pd.NA)
df['kcal'] = df['kcal'].fillna(0)

df['qtde_carboidrato'] = df['qtde_carboidrato'].replace('', pd.NA)
df['qtde_carboidrato'] = df['qtde_carboidrato'].fillna(0)

df['qtde_proteina'] = df['qtde_proteina'].replace('', pd.NA)
df['qtde_proteina'] = df['qtde_proteina'].fillna(0)

df['qtde_gordura'] = df['qtde_gordura'].replace('', pd.NA)
df['qtde_gordura'] = df['qtde_gordura'].fillna(0)

df['qtde_gordura'] = df['qtde_gordura'].replace('', pd.NA)
df['qtde_alcool'] = df['qtde_alcool'].fillna(0)

# (df[['kcal', 'qtde_proteina', 'qtde_gordura', 'qtde_carboidrato', 'qtde_alcool']] == '').sum()

In [70]:
df['nome_alimento'] = df['nome_alimento'].str.replace(r'\s*\(.*?\)\s*', ' ', regex=True)
df['nome_alimento'] = df['nome_alimento'].str.replace(r'\s+', ' ', regex=True).str.strip()
# df.head()

In [71]:
df['nome_alimento'] = df['nome_alimento'].str.replace('não especificada', '', regex=False).str.strip()
df['nome_alimento'] = df['nome_alimento'].str.replace('não especificado', '', regex=False).str.strip()
# df

In [72]:
# print(df.estado_alimento.unique())
df.loc[df['estado_alimento'].str.contains('Não se aplica', case=False, na=False), 'estado_alimento'] = 'PADRAO'
df.loc[df['estado_alimento'].str.contains('Cru', case=False, na=False), 'estado_alimento'] = 'CRU'
df.loc[df['estado_alimento'].str.contains('Cozido', case=False, na=False), 'estado_alimento'] = 'COZIDO'
df.loc[df['estado_alimento'].str.contains('Grelhado', case=False, na=False), 'estado_alimento'] = 'GRELHADO'
df.loc[df['estado_alimento'].str.contains('Assado', case=False, na=False), 'estado_alimento'] = 'ASSADO'
df.loc[df['estado_alimento'].str.contains('Refogado', case=False, na=False), 'estado_alimento'] = 'REFOGADO'
df.loc[df['estado_alimento'].str.contains('Frito', case=False, na=False), 'estado_alimento'] = 'FRITO'

print(df.estado_alimento.unique())

['PADRAO' 'CRU' 'COZIDO' 'GRELHADO' 'ASSADO' 'REFOGADO' 'Ensopado'
 'Mingau' 'Sopa' 'FRITO' 'Molho vermelho' 'Molho branco' 'Ao alho e óleo'
 'Com manteiga/óleo' 'Empanado(a)/à milanesa' 'Ao vinagrete']


In [73]:
valores_validos = ['PADRAO', 'CRU', 'COZIDO', 'GRELHADO', 'ASSADO', 'REFOGADO', 'FRITO']
df = df[df['estado_alimento'].isin(valores_validos)]
print(df.estado_alimento.unique())

['PADRAO' 'CRU' 'COZIDO' 'GRELHADO' 'ASSADO' 'REFOGADO' 'FRITO']


In [74]:
print(df.shape)
df = df.drop_duplicates(subset=['nome_alimento', 'estado_alimento'], keep='first')
print(df.shape)

(1673, 31)
(1671, 31)


In [75]:
print(df.shape)
df = df[~((df['estado_alimento'] == 'PADRAO') & (df['nome_alimento'].duplicated(keep=False)))]
print(df.shape)

(1671, 31)
(1607, 31)


In [76]:
def convert_and_truncate(x):
    try:
        return round(float(x.replace(',', '.')), 1)
    except (ValueError, AttributeError):
        return x

df = df.applymap(convert_and_truncate)

/var/folders/40/ncqy45yx5y59gm95g5v_kx480000gn/T/ipykernel_7229/373288614.py:7: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  df = df.applymap(convert_and_truncate)


In [77]:
df['id_alimento'] = range(1, len(df) + 1)
df['id_tabela_nutricional'] = df['id_alimento']

In [78]:
# colunas_float = valores = [ "id_tabela_nutricional", "kcal", "qtde_proteina", "qtde_carboidrato", "qtde_gordura", "qtde_alcool", "qtde_acucar", "qtde_fibra", "qtde_saturada", "qtde_monosaturada",  "qtde_polisaturada", "qtde_trans", "qtde_sodio", "qtde_calcio", "qtde_ferro", "qtde_potassio", "qtde_vitamina_a", "qtde_vitamina_c", "qtde_vitamina_d", "qtde_vitamina_e"
# ]
# for col in colunas_float:
#     df[col] = pd.to_numeric(df[col], errors='coerce')

# colunas_com_strings_vazias = df.applymap(lambda x: isinstance(x, str) and x == '').any()

# # Exibir o resultado
# print(colunas_com_strings_vazias)

df [df.qtde_gordura.isnull()]['qtde_gordura']

Series([], Name: qtde_gordura, dtype: float64)

In [79]:
df.to_csv('TabelaNutrIA-Tratada.csv', sep=',', index=False)

In [80]:
df_alimentos = df[['id_alimento', 'id_usuario', 'nome_alimento', 'estado_alimento', 'alimento_verificado', 'grupo_alimentar', 'marca_alimento', 'dtt_criacao_alimento', 'alimento_ativo']]
df_alimentos.to_csv('TabelaNutrIA-Alimentos.csv', sep=',', index=False)

In [81]:
colunas_diferenca = df.columns.difference(df_alimentos.columns)

df_tabela_nutricional = df[colunas_diferenca]
df_tabela_nutricional['id_alimento'] = df['id_alimento']
df_tabela_nutricional.to_csv('TabelaNutrIA-TabelaNutricional.csv', sep=',', index=False)

/var/folders/40/ncqy45yx5y59gm95g5v_kx480000gn/T/ipykernel_7229/3474774222.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_tabela_nutricional['id_alimento'] = df['id_alimento']
